# 데이터 불러오기
필요한 패키지를 임포트하고 데이터를 불러와줍니다.

In [ ]:
# (참고) import pandas as pd   * pandas : 데이터 처리, 분석 라이브러리 
#import numpy as np  * numpy : 대규모 다차원 배열, 행렬연산에 필요한 라이브러리 
#import matplotlib.pyplot as plt  * matplotlib : Python 프로그래밍 언어 및 수학적 확장 NumPy 라이브러리를 활용한 플로팅 라이브러리
#import seaborn as sns  * seaborn : 데이터 시각화 담당 라이브러리
#from scipy import stats  * SciPy : 파이썬을 기반으로 하여 과학, 분석, 그리고 엔지니어링을 위한 과학(계산)적 컴퓨팅 영역의 여러 기본적인 작업을 위한 라이브러리(패키지 모음)


import warnings #*warning  라이브러리를 사용해서
warnings.filterwarnings(action='ignore') # 경고 메세지를 무시하고 숨기기

import pandas as pd 

train_data = pd.read_csv("./data/train.csv")  # 훈련 데이터 csv파일 읽어오기
sample_submission = pd.read_csv("./data/sample_submission.csv") # sample_submission.csv 파일 읽어오기

train_data.head() # 훈련 데이터의 앞의 5개의 정보만 출력해서 보여주기 

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
0,1,10,2392,Ex,3,968,Ex,2392,2392,Ex,2,2003,2003,2003,386250
1,2,7,1352,Gd,2,466,Gd,1352,1352,Ex,2,2006,2007,2006,194000
2,3,5,900,TA,1,288,TA,864,900,TA,1,1967,1967,1967,123000
3,4,5,1174,TA,2,576,Gd,680,680,TA,1,1900,2006,2000,135000
4,5,7,1958,Gd,3,936,Gd,1026,1026,Gd,2,2005,2005,2005,250000


출력된 데이터의 모양을 살펴보면 총 15개의 열(column)이 존재하는 것을 확인할 수 있습니다.

15 개 중 2개는 id 와 target 열 입니다.

데이터를 자세히 살펴보는 것은 EDA 글을 통해 더 자세히 배워봅시다.

여기에서는 분석 방법에 초점을 맞추겠습니다.

데이터를 살펴보면 데이터의 모양이 조금씩 다릅니다.

Gr Liv Area, Garage Cars 과 같은 데이터들은 수치를 나타내는 데이터로 수치형(numerical) 데이터라고 합니다.

반면, Kitchen Qual, Bsmt Qual 와 같은 데이터는 수치가 아닌 종류를 나타내죠. 이러한 데이터를 범주형(categorical) 데이터라고 합니다.

범주형 데이터의 라벨이 숫자가 아닌 문자열의 형식으로 저장되어 있다면 데이터 분석 과정에서 인코딩(encoding)이 필요합니다.

## 결측치 확인

결측치(NA: Not Available)란 값이 누락된 데이터를 말합니다.

보다 정확한 분석을 하기 위해서는 데이터의 결측치를 확인하고 적절히 처리해주어야 합니다.

이번 데이터에 결측치가 있나 확인해볼까요?

In [ ]:
def check_missing_col(dataframe): # baselne 1에서와 같이 결측치를 확인하는 함수 
    missing_col = []  
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train_data)

결측치가 존재하지 않습니다


### 라벨 인코딩
라벨인코더를 이용해 라벨인코딩을 진행해주려 합니다.

라벨인코딩이란, 범주형 변수를 숫자로 인코딩해주는 동시에 컴퓨터가 이들이 각각 다른 범주인 것을 인식하도록 각 값을 0, 1, 2 등의 값으로 구별하여 인코딩하는 기법을 말합니다.

라벨 인코딩 함수를 작성해 직접 변수들을 인코딩하겠습니다.

In [ ]:
#라벨 인코딩을 하기 전 불필요한 열을 제거하고, 범주형 데이터 열과 수치형 데이터 열을 나누어줍니다. 
train_x = train_data.drop(["id","target"],axis=1) # 불필요한 데이터 id와 target 열 제거
train_y = train_data.target 

num_features = ['Overall Qual', 'Gr Liv Area', 'Garage Cars',  # 수치형 데이터 열
       'Garage Area', 'Total Bsmt SF', '1st Flr SF',
       'Full Bath', 'Year Built', 'Year Remod/Add',
       'Garage Yr Blt']

cat_features = ["Exter Qual", "Kitchen Qual","Bsmt Qual"] # 범주형 데이터 열
 
from sklearn.preprocessing import LabelEncoder # 라벨인코딩 기능을 사용하기 위해서 sklearn의 preprocessing의 LabelEncoder를 import 
#(사이킷런은 파이썬에서 머신러닝 분석을 할 때 유용하게 사용할 수 있는 라이브러리/ 여러가지 머신러닝 모듈로 구성)

le = LabelEncoder()

for feature in cat_features:
    train_x[feature] = le.fit_transform(train_x[feature])  # train_x[feature]의 데이터를 기준으로 수치화하는 코드이다. 

### 정규화 
수치형 데이터들을 정규화 시켜줍니다.

머신러닝 과정에서 모델은 데이터의 특성(feature)들을 추출해 학습을 진행합니다.

하지만 학습을 하는 과정에서 데이터의 값이 너무 크거나, 분산이 너무 크면 학습 과정에 악영향을 끼칠 수 있습니다. 

따라서 정규화를 통해 데이터 값의 크기를 줄이고 분산을 줄여 모델이 데이터를 이상하게 해석하는 것을 방지합니다.

이번 베이스라인에서는 min-max 정규화를 이용해 봅시다.

min-max 정규화는 수치형 데이터의 값을 0~1 사이의 값으로 변환해줍니다.

min-max 정규화의 수식은 아래와 같습니다.

X' = (X - MIN) / (MAX-MIN)

In [ ]:
#정규화를 위한 패키지를 불러옵니다.
from sklearn.preprocessing import MinMaxScaler

#수치형 데이터를 정규화 시켜줍니다.
scaler = MinMaxScaler()
train_x[num_features] = scaler.fit_transform(train_x[num_features])

### 모델 설계 & 학습

#### GridSearchCV 를 사용한 파라미터 최적화

이제 데이터를 전처리하는 과정이 끝났습니다. 이제 데이터 분석을 위해 모델을 설계하는 단계를 진행해야 합니다.

우선 사용할 모델을 선택하여야 합니다. 사용할 모델은 데이터의 특성, task 의 특성에 알맞게 선택해야 합니다.

모델을 선택했다면 모델의 파라미터를 최적화하는 과정을 거쳐야 합니다. 

파라미터란 모델의 설정을 말합니다. 예를 들어 Linear Regression 모델의 경우 y 절편을 0으로 설정할 수 있습니다.

파라미터 최적화를 하는 기본적인 방법은 여러 파라미터들을 실험하는 방법입니다.

sklearn 은 GridSearchCV 메소드를 제공합니다.

우선 선택한 모델과 비교하고 싶은 파라미터들을 설정해 줍니다. 

그 다음 학습 데이터를 나누어줍니다. 예를 들어 학습 데이터를 5개의 배치로 나누어 주었다고 가정 해보겠습니다.

5개로 나누어진 데이터 중, 4개의 데이터로 학습을 진행합니다. 그 다음 나머지 1개의 데이터 배치로 성능을 검증합니다.

따라서 5번의 학습과 검증이 이루어질 것 입니다. 이때 최상의 성능을 출력하는 파라미터들이 모델의 파라미터로 선택되는 것 입니다.

데이터를 n개의 배치로 나누고, 비교하고자 하는 파라미터들의 조합이 m개라면 총 n * m 번의 학습과 검증이 진행되는 것 입니다.

따라서 학습 시간이 오래 걸리고, 그에 맞는 컴퓨팅 자원이 필요합니다.

이번 베이스라인에서는 GridSearchCV 기법의 개념을 소개하는 방향으로 작성되었습니다.

따라서 앙상블 기법에 사용된 세부적인 모델들의 세부적인 개념들을 추후 베이스라인들에서 소개하겠습니다.

In [ ]:
#GridSearchCV 를 사용할 모델들을 호출합니다.
from sklearn.ensemble import RandomForestRegressor # 랜덤포레스트는 dataset의 다양한 하위 sample에 대한 여러 개의 Decision Tree에서의 평균을 사용하여 predict Accuracy를 높이고 overffing을 제어하는 meta estimator 
from sklearn.ensemble import GradientBoostingRegressor # 앙상블 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가하지만 이전 예측기가 만든 잔여오차에 새로운 예측기를 학습시킴
from sklearn.ensemble import ExtraTreesRegressor # 데이터 세트의 다양한 하위 샘플에 여러 무작위 결정 트리(extra-trees라고도 함)에 맞는 메타 추정기를 구현하고 평균을 사용하여 예측 정확도를 개선하고 과적합을 제어
from sklearn.model_selection import GridSearchCV # GridSearchCV:사이킷런에서는 분류 알고리즘이나 회귀 알고리즘에 사용되는 하이퍼파라미터를 순차적으로 입력해 학습을 하고 측정을 하면서 가장 좋은 파라미터를 알려줌

#모델들을 할당할 리스트를 만들어줍니다.
estimators = []

#estimators 리스트에 모델들을 추가해줍니다.
rf = RandomForestRegressor()
estimators.append(rf)

gbr = GradientBoostingRegressor()
estimators.append(gbr)

etr = ExtraTreesRegressor()
estimators.append(etr)

#모들의 파라미터들을 할당할 리스트를 만들어줍니다.
params = []

# params 리스트에 성능을 비교하고자하는 파라미터들 추가해줍니다.
params_rf = {'n_estimators' : [90, 100, 110, 120],
            'min_samples_split' : [1,2,3,4]}
params.append(params_rf)

params_gbr = {'loss' : ['huber', 'quantile'],
             'learning_rate':[0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15],
             'n_estimators':[60,70,80,90,100,110,120,130,140,150]}
params.append(params_gbr)

params_etr = {'n_estimators' : [50,60,70,80,90,100,110,120,130,140,150]}
params.append(params_etr)

In [ ]:
#GridSearchCV 를 이용해 모델들을 최적화시켜줍니다.
from tqdm.auto import tqdm # 파이썬으로 어떤 작업을 수행 중일 때, 프로그램이 내가 의도한 대로 돌아가고 있는 중인지 확인할 수 있음
def gridSearchCV(models,params):
    best_models=[]
    for i in tqdm(range(0,len(models))):
        model_grid = GridSearchCV(models[i], params[i],n_jobs = -1, verbose=1, cv=5)
        model_grid.fit(train_x,train_y)
        best_models.append(model_grid.best_estimator_)
    return best_models

best_model_list = gridSearchCV(estimators,params)

  0%|          | 0/3 [00:00<?, ?it/s]

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    3.4s finished


Fitting 5 folds for each of 220 candidates, totalling 1100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 1100 out of 1100 | elapsed:   24.2s finished


Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    1.5s finished


In [ ]:
#GridSerachCV 를 통해 최적화된 모델들을 확인합니다.
best_model_list

[RandomForestRegressor(n_estimators=90),
 GradientBoostingRegressor(learning_rate=0.09, loss='huber', n_estimators=150),
 ExtraTreesRegressor(n_estimators=110)]

#### 앙상블 모델 학습

위 코드에서는 3개의 모델을 선택하여 GridSearchCV 를 통해 각 모델들의 파라미터들을 최적화해주었습니다.

그 다음에는 3개의 모델을 모두 활용하여 하나의 모델을 설계해 보겠습니다.

앙살블 기법은 "집단지성" 으로 부터 아이디어를 얻은 알고리즘입니다.
 
모델 하나의 예측 보다는 여러 모델들의 예측을 종합하여 도출한 예측이 더 정확할 때가 많습니다. 

Regression 의 경우 각 모델들이 개별로 예측한 값의 평균 값을 도출하는 방식으로 여러 모델들을 조합해줍니다.

sklearn 패키지의 VotingRegressor 메소드를 통해 앙상블 모델을 구현할 수 있습니다.

GridSearchCV 를 통해 최적의 파라미터들을 찾은 모델들 VotingRegressor 객체에 할당한 후, 학습을 진행합니다.

이번 베이스라인에서는 앙상블 기법의 개념을 소개하는 방향으로 작성되었습니다.

따라서 앙상블 기법에 사용된 세부적인 모델들의 세부적인 개념들을 추후 베이스라인들에서 소개하겠습니다.

In [ ]:
#GridSearchCV 를 통해 최적화된 모델들을 사용합니다.
best_models = [
    ('rf', RandomForestRegressor(min_samples_split=3, n_estimators=120)),
    ('GBR', GradientBoostingRegressor(learning_rate=0.09, loss='huber', n_estimators=130)),
    ('ET', ExtraTreesRegressor(n_estimators=90))
]

#앙상블 기법을 위한 패키지를 불러옵니다.
from sklearn.ensemble import VotingRegressor

#앙상블 모델을 학습시켜줍니다.
voting_rg = VotingRegressor(estimators=best_models)
voting_rg.fit(train_x,train_y)

VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(min_samples_split=3,
                                                   n_estimators=120)),
                            ('GBR',
                             GradientBoostingRegressor(learning_rate=0.09,
                                                       loss='huber',
                                                       n_estimators=130)),
                            ('ET', ExtraTreesRegressor(n_estimators=90))])

### 추론

학습 데이터에 했던 전처리 과정(라벨인코딩, 정규화)를 테스트 데이터에도 해줍니다.

단, 학습 데이터에 학습된 인코더와 정규화 파라미터를 사용해 주어야 합니다. 

(테스트 데이터를 학습 시킨 후 인코딩과 정규화를 진행하면 data leakage 에 해당됩니다!)

In [ ]:
test_data = pd.read_csv("./data/test.csv")

test_x = test_data.drop("id",axis=1)

for feature in cat_features:
    test_x[feature] = le.transform(test_x[feature])
    
test_x[num_features] = scaler.transform(test_x[num_features])

In [ ]:
#학습된 모델을 통해 test 데이터를 추론합니다.
predictions = voting_rg.predict(test_x)

sample_submission.target = predictions
sample_submission.to_csv("submission.csv",index = False)